### https://leetcode.com/problems/best-time-to-buy-and-sell-stock-with-cooldown/

# Algorithm
We will follow the state transition by inaction DP paradigm where one of the actions we allow for our current state is doing nothing. "Doing nothing" refers to moving on to the next state without incurring any cost for moving past the current state. In other words, both the next state and the current state will yield the same exactl result. Let's say that the problem represents our current state and the subproblem represents our next state. Then, we get the same exact answer to both our problem and subproblem.

Top-Down (Memoization) Approach:
State Variables
i is an index into the input list of prices. It's 0-indexed and represents the current day (we typically refer to the ith day).
holding is a boolean variable whether or not we currently possess a unit of stock. This means that we have bought a unit of stock and have not yet sold it. Remember that in order to fully complete a transaction, we must not only buy the stock but also sell it. 
cooldown is also a boolean variable representing whether or not we are currently in a cooldown period. If we have sold a stock on the previous day, we are in a one day cooldown period, meaning we are forced to do nothing. We simply move on to the next state without incurring a cost (i.e. buying a stock or selling a stock) for passing the current day. 

1. A function or data structure to compute/hold the answer to the problem for every state
We will implement a recursive function dp, which will compute the maximum profit we can attain starting from the current state where i is the first day from which we will start accumulating profits (it is used as an index into the list of prices to indicate how much we will lose from buying a stock or how much we will gain from selling a stock). The variable cooldown will enforce whether or not we can perform any action for the ith day. If it is true, we cannot and must do nothing. Lastly, the variable holding influences the control flow for the recurrence relation. If we currently passes a unit of stock (holding == true), the only two actions we must choose from is selling the stock or doing nothing. If we are not holding on to any stocks, the only two actions we must choose from is either buying a stock or doing nothing. Remember that we cannot engage in multiple transactions simultaneously, meaning that we must sell a stock before moving on and buying the next unit of stock. We cannot have more than one unit of stock in our possession at any given moment in time. We will also memoize our intermediate results so that we don't make use of recurrence relaiton and branch our to at most two additional recursive calls even for duplicate inputs. 

2. A recurrence relation to transition between states
If we are currently holding on to a stock, our only two courses of action are either selling the stock or doing nothing. If we are not holding onto a stock, our only two courses of action are either buying a unit of stock or doing nothing. Let's first start with the case where we are in a cooldown period for a day since we sold a unit of stock on the previous day (cooldown == true). We cannot perform any buying or selling operations, so we will simply do nothing. So, the mathematical recurrence relation expression for doing nothing is
dp[i][holding][cooldown] = dp[i+1][holding][0]
We may make use of this experience if we are in a cooldown period, so we must update cooldown to be false for the next state since a cooldown only lasts for a day. Even if we are not currently in a cooldown period and we simply decide to do nothing, the value we were passed in as input for cooldown for our current state won't change as we pass it once again for our next state.
When we buy a stock, we lose money from the profit we have accumulated thus for. The amount of money we lose is the exact price of the stock for the ith day. So, the mathematical expression for buying a stock is 
dp[i][holding][cooldown] = dp[i+1][1][cooldown]
We update holding to true since we just bought a unit of stock, so our next course of action in the next state may be to sell it in order to fully complete the transaction, so we inform the next state that we are currently in possession of a unit of stock by passing true as an argument to it. When we sell a stock, we gain money to add to the profit we have accumulated thus for. The exact amount of money we gain is the price of the stock on the ith day. The mathematical expression for selling is 

dp[i][holding][cooldown] = dp[i+1][0][1]
Once we sell a stock, we are no longer holding onto anything, so we will update to be false for the next state. Since we just said a unit of stock, a pre-day cooldown period is enforced, meaning we cannot perform a buying operation on the next day, so we pass true for the cooldown argument to inform the next recursive call of what happened on the previous day.

3. A base case to stop recursion
If we run out of days (i.e i == length of prices, we cannot make any profit not lose it, so we will return 0. This is similar to saying what if we had no days in the first place, then there would be no way of making any profit. There is no limit on the number of transactions we can make at most, so we must consider the full range of days. 

Bottom-Up Approach (Tabulation):
Our implementation will be very similar to the top-down approach, especially in terms of the recurrence relation, except for a few notable differences. We will maintain a 3d-arrya dp where i is an index into the first dimension of the array (the 2d matrix we retrieve when we index into the first dimension corresponds into the ith day), holding is an index into the second dimension of the array, and cooldown is an index into the third dimension of the array. We will translate our recursive calls from our top-down approach into array lookups in our bottom-up approach. Notice that in the top-down approach, we made recursive calls for the next day (our subproblems) and waited until the base case was hit in order to execute these recursive calls in LIFO order. Only after receiving the subproblem reuslts could we figure our the answer for the current state. We do not have the luxury of waiting until the base case is hit with the bottom-up approach, so inseead we'll start at the base case. We will need to iterate backwards starting form the base case with state variables i in the outer loop. In other words, we will go backwards through the days. Whether or not we make a backward or forward pass through the second and third dimensions doesn't really matter, so let's make a backward pass just to be consistent with our outer loop. The reason the iteration direction doesn't matter is neither the holding state variable. 

The runtime and space complexity analysis is the same for both the top-down and bottom-up implementations
Runtime Analysis: We have a nested for loop will run for N iterations where N = length of the prices. The first inner loop wil run for exactly two iterations (to switch between the true and false values that boolean variables holding can store). The very innermost loop will run once again for exactly two iterations (to switch between the true and false values that boolean variable cooldown can store). This nested for loop dominates the time complexity, so the overall runtime is O(N*2*2). However, we can ignore coeeficients in our Big-O expressions, so this simplifies to O(N). 

Space Complexity: We maintain a 3d arrya dp where the size of the first dimension of the array is N = length of prices, the size of the seocnd dimension is 2 and the size of the third dimension is 2. This is the only auxillary data structure in our bottom-up implementation, so the overall memory usage of both our top-down and bottom-up implementaitons is O(N*2*2), which can once again be simplified down to O(N). However, we can optimize this space complexity further with state compression. Notice that at any particular moment in time, we don't really need a 2d matrix for the current day and another 2d matrix for the next day. Anytime we encounter dp[i] in our bottom-up approach, we will translate it to a lookup from the current day list. Likewise, when we encounter dp[i+1], we will translate it to a lookup form the next day list. Therefore our overall memory usage will reduce down to O(2*2) or O(1) constant space usage since the size of the inputs don't have an affect on our overall memory consumption. We can have a different input with a different size input prices list, but the size of the lists corresponding to the current day and next day will not change. 

In [ ]:
#Top-Down Approach (Memoization)
class Solution:
    def dp(self,i,holding,cooldown,prices,cache):
        if i == len(prices):
            return 0
        if (i,holding,cooldown) in cache:
            return cache[(i,holding,cooldown)]
        if cooldown:
            cache[(i,holding,cooldown)] = self.dp(i+1,holding,0,prices,cache)
        elif not holding:
            cache[(i,holding,cooldown)] = max(self.dp(i+1,holding,cooldown,prices,cache),self.dp(i+1,1,cooldown,prices,cache)- prices[i]) 
            
        else:
            cache[(i,holding,cooldown)] = max(self.dp(i+1,holding,cooldown,prices,cache),self.dp(i+1,0,1,prices,cache)+ prices[i]) 
        return cache[(i,holding,cooldown)]
    
    def maxProfit(self, prices: List[int]) -> int:
        cache = dict()
        return self.dp(0,False,False,prices,cache)

In [ ]:
class Solution:    
    def maxProfit(self, prices: List[int]) -> int:
        dp = [[ [0 for col in range(2)] for col in range(2)] for row in range(len(prices)+1)]
        for i in range(len(prices)-1,-1,-1):
            for holding in range(1,-1,-1):
                for cooldown in range(1,-1,-1):
                    if cooldown:
                        dp[i][holding][cooldown] = dp[i+1][holding][0]
                    elif holding:
                        dp[i][holding][cooldown] = max(dp[i+1][holding][0],dp[i+1][0][1] +prices[i])
                    else:
                        dp[i][holding][cooldown] = max(dp[i+1][holding][cooldown],dp[i+1][1][cooldown] - prices[i])
        return dp[0][0][0]

In [ ]:
#State-Compression
class Solution:    
    def maxProfit(self, prices: List[int]) -> int:
        curr_day = []
        next_day = []
        for i in range(2):
            list1 = []
            list2= []
            for j in range(2):
                list1.append(0)
                list2.append(0)
            curr_day.append(list1)
            next_day.append(list2)
            
        
        for i in range(len(prices)-1,-1,-1):
            for holding in range(1,-1,-1):
                for cooldown in range(1,-1,-1):
                    if cooldown:
                        curr_day[holding][cooldown] = next_day[holding][0]
                    elif holding:
                        curr_day[holding][cooldown] = max(next_day[holding][0],next_day[0][1] +prices[i])
                    else:
                        curr_day[holding][cooldown] = max(next_day[holding][cooldown],next_day[1][cooldown] - prices[i])
            next_day,curr_day = curr_day,next_day
        return next_day[0][0]
    